### **Proyecto 2. Análisis de la Ejecución de Ingresos Públicos en Brasil**

- Librerías y configuración

In [11]:
# Para tratamiento de datos
import numpy as np
import pandas as pd
# Para visualizaciones
import seaborn as sns 
import matplotlib.pyplot as plt
# Para generar todas las combinaciones posibles
import itertools
# Para guardar DataFrames en Excel
from pandas import ExcelWriter
# Para gestión de fechas
from datetime import datetime
# Ignorar warnings
import warnings
warnings.filterwarnings("ignore")
# Configuración para poder visualizar todas las columnas de los DataFrames
pd.set_option('display.max_columns', None) 

- Función que genera un reporte de nulos en el dataframe

In [21]:
# definimos una función que nos genere un reporte
def reporte(df):
    df_reporte = pd.DataFrame()
    df_reporte["valores_nulos"] = df.isnull().sum()
    df_reporte["porcentaje_nulos"] = round(((df.isnull().sum())/df.shape[0])*100,2)
    df_reporte["tipo_dato"]=df.dtypes
    return df_reporte

### Fase 1: Unión de Conjuntos de Datos

- Carga de archivos

In [44]:
# cargamos los archivos
# tenemos que añadir el sep = ";" porque los archivos están separados por ; y no por ,
# añadimos el encoding = "latin-1" porque como es un archivo en portugués, pandas no lo lee bien
df_2013 = pd.read_csv("datos/datos-2013.csv", sep = ";", encoding = "latin-1")
df_2014 = pd.read_csv("datos/datos-2014.csv", sep = ";", encoding = "latin-1")
df_2015 = pd.read_csv("datos/datos-2015.csv", sep = ";", encoding = "latin-1")
df_2016 = pd.read_csv("datos/datos-2016.csv", sep = ";", encoding = "latin-1")
df_2017 = pd.read_csv("datos/datos-2017.csv", sep = ";", encoding = "latin-1")
df_2018 = pd.read_csv("datos/datos-2018.csv", sep = ";", encoding = "latin-1")
df_2019 = pd.read_csv("datos/datos-2019.csv", sep = ";", encoding = "latin-1")
df_2020 = pd.read_csv("datos/datos-2020.csv", sep = ";", encoding = "latin-1")
df_2021 = pd.read_csv("datos/datos-2021.csv", sep = ";", encoding = "latin-1")

- Exploramos cada archivo individualmente

In [112]:
df_2013.shape

(4498, 16)

In [46]:
reporte(df_2013)

,valores_nulos,porcentaje_nulos,tipo_dato
CÓDIGO ÓRGÃO SUPERIOR,0,0.0,int64
NOME ÓRGÃO SUPERIOR,0,0.0,object
CÓDIGO ÓRGÃO,0,0.0,int64
NOME ÓRGÃO,0,0.0,object
CÓDIGO UNIDADE GESTORA,0,0.0,int64
NOME UNIDADE GESTORA,0,0.0,object
CATEGORIA ECONÔMICA,0,0.0,object
ORIGEM RECEITA,0,0.0,object
ESPÉCIE RECEITA,0,0.0,object
DETALHAMENTO,0,0.0,object


No hay valores nulos y la mayoría de las columnas son de tipo object. Esto no acaba de encajar con el tipo de datos que deberían contener las columnas, es posible que las columnas de VALOR PREVISTO ATUALIZADO', 'VALOR LANÇADO', 'VALOR REALIZADO' y 'PERCENTUAL REALIZADO' deban ser de tipo numérico y 'DATA LANÇAMENTO' de tipo fecha
Pasamos a hacer un describe de las columnas tipo object para tener un recuento de valores únicos

In [48]:
df_2013.describe(include ="O").T

,count,unique,top,freq
NOME ÓRGÃO SUPERIOR,4498,25,Ministério da Educação,1833
NOME ÓRGÃO,4498,266,Ministério da Economia - Unidades com vínculo ...,494
NOME UNIDADE GESTORA,4498,279,SETORIAL ORCAMENTARIA E FINANCEIRA / ME,415
CATEGORIA ECONÔMICA,4498,5,Receitas Correntes,4001
ORIGEM RECEITA,4498,15,Outras Receitas Correntes,1982
ESPÉCIE RECEITA,4498,44,Receita de Serviços,978
DETALHAMENTO,4498,704,RECUPERACAO DE DESPESAS DE EXERC. ANTERIORES,214
VALOR PREVISTO ATUALIZADO,4498,2788,"0,00",1625
VALOR LANÇADO,4498,52,"0,00",4447
VALOR REALIZADO,4498,3738,"0,00",704


In [49]:
df_2013.describe().T

,count,mean,std,min,25%,50%,75%,max
CÓDIGO ÓRGÃO SUPERIOR,4498.0,31729.657626,10691.972258,20000.0,25000.0,26000.0,36000.0,81000.0
CÓDIGO ÓRGÃO,4498.0,31447.970431,10798.289531,20101.0,25916.0,26292.0,35000.0,91214.0
CÓDIGO UNIDADE GESTORA,4498.0,233059.797910,141561.057724,110005.0,153173.0,160075.0,254420.0,913001.0
ANO EXERCÍCIO,4498.0,2013.000000,0.000000,2013.0,2013.0,2013.0,2013.0,2013.0


El describe() de las columnas numéricas en este caso no aporta nada, ya que los códigos son identificadores de cada organización y el año es el mismo en todos los registros.

In [113]:
df_2014.shape

(4553, 16)

In [53]:
reporte(df_2014)

,valores_nulos,porcentaje_nulos,tipo_dato
CÓDIGO ÓRGÃO SUPERIOR,0,0.0,int64
NOME ÓRGÃO SUPERIOR,0,0.0,object
CÓDIGO ÓRGÃO,0,0.0,int64
NOME ÓRGÃO,0,0.0,object
CÓDIGO UNIDADE GESTORA,0,0.0,int64
NOME UNIDADE GESTORA,0,0.0,object
CATEGORIA ECONÔMICA,0,0.0,object
ORIGEM RECEITA,0,0.0,object
ESPÉCIE RECEITA,0,0.0,object
DETALHAMENTO,0,0.0,object


In [54]:
df_2014.describe(include="O").T

,count,unique,top,freq
NOME ÓRGÃO SUPERIOR,4553,25,Ministério da Educação,1841
NOME ÓRGÃO,4553,267,Ministério da Economia - Unidades com vínculo ...,482
NOME UNIDADE GESTORA,4553,278,SETORIAL ORCAMENTARIA E FINANCEIRA / ME,382
CATEGORIA ECONÔMICA,4553,5,Receitas Correntes,4095
ORIGEM RECEITA,4553,15,Outras Receitas Correntes,2050
ESPÉCIE RECEITA,4553,46,Receita de Serviços,973
DETALHAMENTO,4553,708,RECUPERACAO DE DESPESAS DE EXERC. ANTERIORES,216
VALOR PREVISTO ATUALIZADO,4553,2740,"0,00",1706
VALOR LANÇADO,4553,72,"0,00",4482
VALOR REALIZADO,4553,3726,"0,00",757


In [114]:
df_2015.shape

(4523, 16)

In [56]:
reporte(df_2015)

,valores_nulos,porcentaje_nulos,tipo_dato
CÓDIGO ÓRGÃO SUPERIOR,0,0.0,int64
NOME ÓRGÃO SUPERIOR,0,0.0,object
CÓDIGO ÓRGÃO,0,0.0,int64
NOME ÓRGÃO,0,0.0,object
CÓDIGO UNIDADE GESTORA,0,0.0,int64
NOME UNIDADE GESTORA,0,0.0,object
CATEGORIA ECONÔMICA,0,0.0,object
ORIGEM RECEITA,0,0.0,object
ESPÉCIE RECEITA,0,0.0,object
DETALHAMENTO,0,0.0,object


In [57]:
df_2015.describe(include="O").T

,count,unique,top,freq
NOME ÓRGÃO SUPERIOR,4523,25,Ministério da Educação,1803
NOME ÓRGÃO,4523,271,Ministério da Economia - Unidades com vínculo ...,481
NOME UNIDADE GESTORA,4523,288,SETORIAL ORCAMENTARIA E FINANCEIRA / ME,385
CATEGORIA ECONÔMICA,4523,4,Receitas Correntes,4088
ORIGEM RECEITA,4523,14,Outras Receitas Correntes,2076
ESPÉCIE RECEITA,4523,41,"Multas administrativas, contratuais e judicia",962
DETALHAMENTO,4523,707,RECUPERACAO DE DESPESAS DE EXERC. ANTERIORES,225
VALOR PREVISTO ATUALIZADO,4523,2648,"0,00",1785
VALOR LANÇADO,4523,83,"0,00",4441
VALOR REALIZADO,4523,3744,"0,00",711


In [115]:
df_2016.shape

(194533, 16)

In [59]:
reporte(df_2016)

,valores_nulos,porcentaje_nulos,tipo_dato
CÓDIGO ÓRGÃO SUPERIOR,0,0.00,int64
NOME ÓRGÃO SUPERIOR,0,0.00,object
CÓDIGO ÓRGÃO,0,0.00,int64
NOME ÓRGÃO,0,0.00,object
CÓDIGO UNIDADE GESTORA,0,0.00,int64
NOME UNIDADE GESTORA,0,0.00,object
CATEGORIA ECONÔMICA,0,0.00,object
ORIGEM RECEITA,0,0.00,object
ESPÉCIE RECEITA,0,0.00,object
DETALHAMENTO,0,0.00,object


In [64]:
df_2016.describe(include="O").T

,count,unique,top,freq
NOME ÓRGÃO SUPERIOR,194533,25,Ministério da Educação,71840
NOME ÓRGÃO,194533,269,Ministério da Economia - Unidades com vínculo ...,13749
NOME UNIDADE GESTORA,194533,293,SETORIAL ORCAMENTARIA E FINANCEIRA / ME,11126
CATEGORIA ECONÔMICA,194533,4,Receitas Correntes,186687
ORIGEM RECEITA,194533,14,Outras Receitas Correntes,70647
ESPÉCIE RECEITA,194533,44,Serviços Administrativos e Comerciais Gerais,54941
DETALHAMENTO,194533,525,SERV.ADMINISTRAT.E COMERCIAIS GERAIS-PRINC.,30237
VALOR PREVISTO ATUALIZADO,194533,1835,"0,00",192639
VALOR LANÇADO,194533,1003,"0,00",193450
VALOR REALIZADO,194533,129122,"0,00",2972


In [116]:
df_2017.shape

(190479, 16)

In [63]:
reporte(df_2017)

,valores_nulos,porcentaje_nulos,tipo_dato
CÓDIGO ÓRGÃO SUPERIOR,0,0.00,int64
NOME ÓRGÃO SUPERIOR,0,0.00,object
CÓDIGO ÓRGÃO,0,0.00,int64
NOME ÓRGÃO,0,0.00,object
CÓDIGO UNIDADE GESTORA,0,0.00,int64
NOME UNIDADE GESTORA,0,0.00,object
CATEGORIA ECONÔMICA,0,0.00,object
ORIGEM RECEITA,0,0.00,object
ESPÉCIE RECEITA,0,0.00,object
DETALHAMENTO,0,0.00,object


In [81]:
df_2017.describe(include="O").T

,count,unique,top,freq
NOME ÓRGÃO SUPERIOR,190479,24,Ministério da Educação,70837
NOME ÓRGÃO,190479,268,Ministério da Economia - Unidades com vínculo ...,14158
NOME UNIDADE GESTORA,190479,318,SETORIAL ORCAMENTARIA E FINANCEIRA / ME,11227
CATEGORIA ECONÔMICA,190479,5,Receitas Correntes,182668
ORIGEM RECEITA,190479,15,Outras Receitas Correntes,68818
ESPÉCIE RECEITA,190479,47,Serviços Administrativos e Comerciais Gerais,51633
DETALHAMENTO,190479,516,SERV.ADMINISTRAT.E COMERCIAIS GERAIS-PRINC.,29841
VALOR PREVISTO ATUALIZADO,190479,1827,"0,00",188589
VALOR LANÇADO,190479,1043,"0,00",189434
VALOR REALIZADO,190479,128532,"0,00",2800


In [117]:
df_2018.shape

(173944, 16)

In [84]:
reporte(df_2018)

,valores_nulos,porcentaje_nulos,tipo_dato
CÓDIGO ÓRGÃO SUPERIOR,0,0.00,int64
NOME ÓRGÃO SUPERIOR,0,0.00,object
CÓDIGO ÓRGÃO,0,0.00,int64
NOME ÓRGÃO,0,0.00,object
CÓDIGO UNIDADE GESTORA,0,0.00,int64
NOME UNIDADE GESTORA,0,0.00,object
CATEGORIA ECONÔMICA,0,0.00,object
ORIGEM RECEITA,0,0.00,object
ESPÉCIE RECEITA,0,0.00,object
DETALHAMENTO,0,0.00,object


In [85]:
df_2018.describe(include="O").T

,count,unique,top,freq
NOME ÓRGÃO SUPERIOR,173944,22,Ministério da Educação,65876
NOME ÓRGÃO,173944,260,Ministério da Economia - Unidades com vínculo ...,22844
NOME UNIDADE GESTORA,173944,312,SETORIAL ORCAMENTARIA E FINANCEIRA / ME,21206
CATEGORIA ECONÔMICA,173944,5,Receitas Correntes,165585
ORIGEM RECEITA,173944,15,Receita de Serviços,58581
ESPÉCIE RECEITA,173944,47,Serviços Administrativos e Comerciais Gerais,51444
DETALHAMENTO,173944,608,SERV.ADMINISTRAT.E COMERCIAIS GERAIS-PRINC.,29257
VALOR PREVISTO ATUALIZADO,173944,1846,"0,00",172042
VALOR LANÇADO,173944,828,"0,00",173114
VALOR REALIZADO,173944,121803,"0,00",2715


In [118]:
df_2019.shape

(176828, 16)

In [87]:
reporte(df_2019)

,valores_nulos,porcentaje_nulos,tipo_dato
CÓDIGO ÓRGÃO SUPERIOR,0,0.00,int64
NOME ÓRGÃO SUPERIOR,0,0.00,object
CÓDIGO ÓRGÃO,0,0.00,int64
NOME ÓRGÃO,0,0.00,object
CÓDIGO UNIDADE GESTORA,0,0.00,int64
NOME UNIDADE GESTORA,0,0.00,object
CATEGORIA ECONÔMICA,0,0.00,object
ORIGEM RECEITA,0,0.00,object
ESPÉCIE RECEITA,0,0.00,object
DETALHAMENTO,0,0.00,object


In [88]:
df_2019.describe(include="O").T

,count,unique,top,freq
NOME ÓRGÃO SUPERIOR,176828,20,Ministério da Educação,67104
NOME ÓRGÃO,176828,249,Ministério da Economia - Unidades com vínculo ...,23514
NOME UNIDADE GESTORA,176828,269,SETORIAL ORCAMENTARIA E FINANCEIRA / ME,22389
CATEGORIA ECONÔMICA,176828,5,Receitas Correntes,169210
ORIGEM RECEITA,176828,15,Receita de Serviços,59957
ESPÉCIE RECEITA,176828,46,Serviços Administrativos e Comerciais Gerais,52747
DETALHAMENTO,176828,636,SERV.ADMINISTRAT.E COMERCIAIS GERAIS-PRINC.,29143
VALOR PREVISTO ATUALIZADO,176828,1801,"0,00",174975
VALOR LANÇADO,176828,1105,"0,00",175649
VALOR REALIZADO,176828,124002,"0,00",3107


In [119]:
df_2020.shape

(142348, 16)

In [91]:
reporte(df_2020)

,valores_nulos,porcentaje_nulos,tipo_dato
CÓDIGO ÓRGÃO SUPERIOR,0,0.00,int64
NOME ÓRGÃO SUPERIOR,0,0.00,object
CÓDIGO ÓRGÃO,0,0.00,int64
NOME ÓRGÃO,0,0.00,object
CÓDIGO UNIDADE GESTORA,0,0.00,int64
NOME UNIDADE GESTORA,0,0.00,object
CATEGORIA ECONÔMICA,0,0.00,object
ORIGEM RECEITA,0,0.00,object
ESPÉCIE RECEITA,0,0.00,object
DETALHAMENTO,0,0.00,object


In [92]:
df_2020.describe(include="O").T

,count,unique,top,freq
NOME ÓRGÃO SUPERIOR,142348,20,Ministério da Educação,40888
NOME ÓRGÃO,142348,254,Ministério da Economia - Unidades com vínculo ...,22686
NOME UNIDADE GESTORA,142348,270,SETORIAL ORCAMENTARIA E FINANCEIRA / ME,21074
CATEGORIA ECONÔMICA,142348,5,Receitas Correntes,135172
ORIGEM RECEITA,142348,15,Outras Receitas Correntes,42127
ESPÉCIE RECEITA,142348,45,Serviços Administrativos e Comerciais Gerais,33271
DETALHAMENTO,142348,625,SERV.ADMINISTRAT.E COMERCIAIS GERAIS-PRINC.,21029
VALOR PREVISTO ATUALIZADO,142348,1951,"0,00",140352
VALOR LANÇADO,142348,1579,"0,00",140661
VALOR REALIZADO,142348,106992,"0,00",3837


In [120]:
df_2021.shape

(134593, 16)

In [94]:
reporte(df_2021)

,valores_nulos,porcentaje_nulos,tipo_dato
CÓDIGO ÓRGÃO SUPERIOR,0,0.00,int64
NOME ÓRGÃO SUPERIOR,0,0.00,object
CÓDIGO ÓRGÃO,0,0.00,int64
NOME ÓRGÃO,0,0.00,object
CÓDIGO UNIDADE GESTORA,0,0.00,int64
NOME UNIDADE GESTORA,0,0.00,object
CATEGORIA ECONÔMICA,0,0.00,object
ORIGEM RECEITA,0,0.00,object
ESPÉCIE RECEITA,0,0.00,object
DETALHAMENTO,0,0.00,object


In [95]:
df_2021.describe(include="O").T

,count,unique,top,freq
NOME ÓRGÃO SUPERIOR,134593,20,Ministério da Educação,39869
NOME ÓRGÃO,134593,256,Ministério da Economia - Unidades com vínculo ...,21403
NOME UNIDADE GESTORA,134593,268,SETORIAL ORCAMENTARIA E FINANCEIRA / ME,20277
CATEGORIA ECONÔMICA,134593,5,Receitas Correntes,128090
ORIGEM RECEITA,134593,15,Receita de Serviços,39917
ESPÉCIE RECEITA,134593,42,Serviços Administrativos e Comerciais Gerais,34121
DETALHAMENTO,134593,600,SERV.ADMINISTRAT.E COMERCIAIS GERAIS-PRINC.,19532
VALOR PREVISTO ATUALIZADO,134593,1803,"0,00",132746
VALOR LANÇADO,134593,1191,"0,00",133403
VALOR REALIZADO,134593,99376,"0,00",3404


Los 3 primeros años (2013, 2014 y 2015) no tienen valores nulos, sin embargo, del 2016 en adelante encontramos valores nulos en la columna "DATA LANÇAMENTO", son pocos, entre el 0.04 y el 0.09 %.
Mientras que en los 3 primero años esta fecha era la misma en todos los registros, a partir de 2016 hay diferentes valores de fecha por lo que se plantea el problema de cómo rellenar esos valores en el caso de que sea necesario. 
Todos los archivos tienen el mismo número de columnas, con el mismo nombre y en el mismo orden, por lo que 